In [1]:
### This source code is for testing with MNIST dataset. 
### Reference: MMD-Aggregated test

%load_ext autoreload 
%autoreload 2

import numpy as np
import itertools
import pandas as pd
import torch
from mnist import download_mnist, load_mnist
from pathlib import Path
from seed import generate_seed
from sample_test import sample_and_test_uniform, sample_and_test_mnist
from tests import TST_C2ST, TST_MMD, TST_MMD_AGG
from models import NeuralNet, MyModel, MNISTResNet
import torchvision.models as models

/home/oldrain123/miniconda3/envs/c2st/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create results directory if it does not exist 
Path("./data").mkdir(exist_ok=True, parents=True) 

# Pandas dataframe: lists of indices and entries 
index_vals = [] 
results = []

In [3]:
# Parameters shared for all experiments 
alpha = 0.05 
B1 = 500 
B2 = 500 
B3 = 100 
kernel_types = ['gaussian', 'laplace']
k_num = len(kernel_types) 

In [4]:
#### MNIST ##### 
# parameters for all mnist experiments 
p_num = 5
d = None 
Q_list = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5'] 
bandwidth_multipliers = np.array([2 ** i for i in range(10, 21)]) 
Path('/home/oldrain123/C2ST/data/MNIST').mkdir(exist_ok=True) 
if Path('/home/oldrain123/C2ST/data/MNIST/mnist_7x7.data').is_file() == False:
    download_mnist() 
P, Q_list = load_mnist() 
Q_list_str = ["Q1", "Q2", "Q3", "Q4", "Q5"] 
                                                        

In [5]:
exp = "8: mnist alternative" 
repetitions = 50
sample_size_m = 100 
sample_sizes_n = [100, 200, 400, 600, 800 ,1000] 
v_num = len(sample_sizes_n) 
function_types = ['uniform', 'increasing', 'decreasing', 'centred', 'ost']
f_num = len(function_types) 
L = [(10, 14), (12, 16)] 
approx_type = "permutation"

In [6]:
n_epochs = [100, 500, 1000] 
e_num = len(n_epochs)
learning_rates = [0.0002, 0.0005, 0.001]
lr_num = len(learning_rates) 
# hidden_neurons = [100, 300, 500]
# h_num = len(hidden_neurons)

In [7]:
device = 'cuda:5'
dtype = torch.float

In [8]:
for e, r, v in itertools.product(range(e_num), range(lr_num), range(v_num)):
    n_epoch = n_epochs[e] 
    n = sample_sizes_n[v] 
    m = sample_size_m
    learning_rate = learning_rates[r] 
    Qi = Q_list[2] 
    perturbation_or_Qi = Q_list_str[2] 
    test_output_list = [] 
    rs = np.random.RandomState(1203 + e + r + v)
    idx_X = rs.randint(len(P), size = n)
    idx_Y = rs.randint(len(Qi), size = m)
    s1 = torch.tensor(P[idx_X, :]).view(-1, 1, 7, 7).to(device, dtype)
    s2 = torch.tensor(Qi[idx_Y, :]).view(-1, 1, 7, 7).to(device, dtype)

    for i in range(repetitions):
        seed = generate_seed(e, 2, 3, r, v, i) 
        # Instantiate the model
        base_model = models.resnet18(pretrained=False)  # or models.resnet34()
        model = MNISTResNet(base_model, in_channels=1, img_size=7, device=device, dtype=dtype)
        test_output_list.append(
            TST_C2ST(
                s1, 
                s2, 
                model,
                alpha, 
                learning_rate, 
                n_epoch, 
                seed, 
                loss_fn = model.smooth_objective,
                device = device 
            )
        )
    
    power = np.mean(test_output_list) 
    index_val = (
        exp, 
        d, 
        repetitions,
        m, 
        n, 
        n_epoch, 
        learning_rate, 
        perturbation_or_Qi, 
    )
    index_vals.append(index_val) 
    results.append(power) 
    print(power) 

print("Experiment completed")

/home/oldrain123/miniconda3/envs/c2st/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 100, Training Stats: 0.3757, Training Accuracy: 0.9500
stats: -0.6461, tau0: 0.4615, tau1: 0.6098, accuracy: 0.4625
Epoch: 100, Training Stats: 0.1933, Training Accuracy: 0.8833
stats: -0.057, tau0: 0.4412, tau1: 0.5652, accuracy: 0.4875
Epoch: 100, Training Stats: 0.2947, Training Accuracy: 0.9250
stats: 0.6905, tau0: 0.35, tau1: 0.575, accuracy: 0.5375
Epoch: 100, Training Stats: 0.3237, Training Accuracy: 0.9333
stats: 0.0, tau0: 0.425, tau1: 0.575, accuracy: 0.5
Epoch: 100, Training Stats: 0.2867, Training Accuracy: 0.9333
stats: 0.8777, tau0: 0.2955, tau1: 0.6111, accuracy: 0.5625
Epoch: 100, Training Stats: 0.2623, Training Accuracy: 0.9083
stats: 0.0168, tau0: 0.5366, tau1: 0.4615, accuracy: 0.5
Epoch: 100, Training Stats: 0.4586, Training Accuracy: 0.9667
stats: 0.2128, tau0: 0.4762, tau1: 0.5, accuracy: 0.5125
Epoch: 100, Training Stats: 0.3212, Training Accuracy: 0.9333
stats: 2.3133, tau0: 0.3684, tau1: 0.381, accuracy: 0.625
Epoch: 100, Training Stats: 0.3040, Traini

/home/oldrain123/C2ST/models.py:665: RuntimeWarning: invalid value encountered in double_scalars
  obj = (1 - tau0 - tau1) / np.sqrt((tau0 * (1 - tau0) / n0) + (tau1 * (1 - tau1) / n1))


Epoch: 100, Training Stats: 0.0021, Training Accuracy: 0.6870
stats: -0.0528, tau0: 0.3449, tau1: 0.6591, accuracy: 0.6166666666666667
Epoch: 100, Training Stats: 0.0074, Training Accuracy: 0.6037
stats: 2.9071, tau0: 0.4403, tau1: 0.3333, accuracy: 0.5722222222222222
Epoch: 100, Training Stats: 0.0110, Training Accuracy: 0.7815
stats: 1.4742, tau0: 0.2236, tau1: 0.6579, accuracy: 0.7305555555555555
Epoch: 100, Training Stats: 0.0059, Training Accuracy: 0.6630
stats: 2.9259, tau0: 0.4119, tau1: 0.3571, accuracy: 0.5944444444444444
Epoch: 100, Training Stats: 0.0050, Training Accuracy: 0.5000
stats: 0.8325, tau0: 0.5674, tau1: 0.3659, accuracy: 0.45555555555555555
0.06
Epoch: 100, Training Stats: 0.0082, Training Accuracy: 0.5424
stats: 0.153, tau0: 0.5693, tau1: 0.4186, accuracy: 0.44545454545454544
Epoch: 100, Training Stats: 0.0065, Training Accuracy: 0.5515
stats: 2.0003, tau0: 0.5076, tau1: 0.3409, accuracy: 0.509090909090909
Epoch: 100, Training Stats: 0.0065, Training Accuracy: 0

In [ ]:
results